In [ ]:
!pip install coarij
!coarij download --kind E --year 2014
!coarij download --kind E --year 2015
!coarij download --kind E --year 2016
!coarij download --kind E --year 2017
!coarij download --kind E --year 2018
!mv data/interim data/CoARiJ

In [1]:
from collections import Counter # データ保管用
import glob  # 指定ディレクトリ以下のファイルを得る

import spacy
from tqdm import tqdm  # プログラスバーは表示

# CoARiJの2014データが保存されているディレクトリ
coarij_dir = 'data/CoARiJ/2014/docs'

# 解析の対象とする品詞
noun_tags = set(['NOUN', 'PRON', 'PROPN'])

# spacyのインスタンスを生成
nlp = spacy.load('ja_ginza')

In [2]:
def chunk(lst, n):
    """
    リストをn個ずつのサブリストに分割する。
    """
    return [lst[i:i+n] for i in range(0, len(lst), n)]


def extract_words(doc, pos_tags):
    words = [token.text.strip()  for token in doc
             if token.pos_ in pos_tags]
    return words


def parse_file(path, nlp, pos_tags):
    words_all = []
    with open(path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            list_lines = line.split("。")
            lst_texts = chunk(list_lines,20)
            for text in lst_texts:
                doc = nlp.tokenizer(" ".join(text))
                words = extract_words(doc, pos_tags)
                words_all.extend(words)
    return words_all

In [3]:
# ファイル一覧を得る
paths = glob.glob(f'{coarij_dir}/**/**.txt', recursive=True)

# 語の頻度を数えるための辞書型オブジェクト
counter = Counter()

# 単語の数を数える
for path in tqdm(paths):
    words = parse_file(path, nlp, noun_tags)
    counter.update(words)

100%|█████████████████████████████████████████████████████████████████████████████| 40563/40563 [15:03<00:00, 44.91it/s]


In [4]:
# 頻度の高い語を上位20個表示
for i, (word, count) in enumerate(counter.most_common(20)):
    print("{:d}, {}, {:d}".format(i+1, word, count))

1, 年, 583237
2, 月, 581580
3, 当社, 453781
4, 平成, 413175
5, 円, 328896
6, 等, 287672
7, 事業, 246475
8, 連結, 208030
9, 取締役, 200894
10, 株式会社, 157455
11, こと, 156446
12, グループ, 154067
13, ため, 142241
14, 監査役, 124526
15, セグメント, 121444
16, 状況, 115423
17, 昭和, 110481
18, 株式, 108566
19, 日, 105759
20, 経営, 105481
